In [1]:
# A join brings together two sets of data, the left and the right, by comparing the value of one or more keys of the left and right and evaluating the result of a join expression that determines whether Spark should bring together the left set of data with the right set of data.

# The join expression determines whether two rows should join, the join type determines what should be in the result set. There are a variety of different join types available in Spark for you to use:

# Inner joins (keep rows with keys that exist in the left and right datasets)

# Outer joins (keep rows with keys in either the left or right datasets)

# Left outer joins (keep rows with keys in the left dataset)

# Right outer joins (keep rows with keys in the right dataset)

# Left semi joins (keep the rows in the left, and only the left, dataset where the key appears in the right dataset)

# Left anti joins (keep the rows in the left, and only the left, dataset where they do not appear in the right dataset)

# Natural joins (perform a join by implicitly matching the columns between the two datasets with the same names)

# Cross (or Cartesian) joins (match every row in the left dataset with every row in the right dataset)

In [2]:
# Let’s create some simple datasets that we can use in our examples:
# Person DataFrame using lists.
person = spark.createDataFrame([
    (0, "Bill Chambers", 0, [100]),
    (1, "Matei Zaharia", 1, [500, 250, 100]),
    (2, "Michael Armbrust", 1, [250, 100])])\
  .toDF("id", "name", "graduate_program", "spark_status")
# Graduate program DataFrame using lists.
graduateProgram = spark.createDataFrame([
    (0, "Masters", "School of Information", "UC Berkeley"),
    (2, "Masters", "EECS", "UC Berkeley"),
    (1, "Ph.D.", "EECS", "UC Berkeley")])\
  .toDF("id", "degree", "department", "school")
# Status DataFrame using lists.
sparkStatus = spark.createDataFrame([
    (500, "Vice President"),
    (250, "PMC Member"),
    (100, "Contributor")])\
  .toDF("id", "status")

In [3]:
# Let’s register these as tables so that we use them throughout this notebook:
person.createOrReplaceTempView("person")
graduateProgram.createOrReplaceTempView("graduateProgram")
sparkStatus.createOrReplaceTempView("sparkStatus")

In [4]:
# Inner joins evaluate the keys in both of the DataFrames or tables and include (and join together) only the rows that evaluate to true.

# In the following example, we join the graduateProgram DataFrame with the person DataFrame to create a new DataFrame:

joinExpression = person["graduate_program"] == graduateProgram['id']

# Keys that do not exist in both DataFrames will not show in the resulting DataFrame.

In [5]:
# Inner joins are the default join, so we just need to specify our left DataFrame and join the right in the JOIN expression:

person.join(graduateProgram, joinExpression).show(5)

+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
 id| name|graduate_program| spark_status| id| degree| department| school|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
 0| Bill Chambers| 0| [100]| 0|Masters|School of Informa...|UC Berkeley|
 1| Matei Zaharia| 1|[500, 250, 100]| 1| Ph.D.| EECS|UC Berkeley|
 2|Michael Armbrust| 1| [250, 100]| 1| Ph.D.| EECS|UC Berkeley|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+

In [6]:
# We can also specify this explicitly by passing in a third parameter, the joinType:
joinType = "inner"

In [7]:
# Performing the join, specifying the joint Type. Default is "inner" join.
person.join(graduateProgram, joinExpression, joinType).show(5)

+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
 id| name|graduate_program| spark_status| id| degree| department| school|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
 0| Bill Chambers| 0| [100]| 0|Masters|School of Informa...|UC Berkeley|
 1| Matei Zaharia| 1|[500, 250, 100]| 1| Ph.D.| EECS|UC Berkeley|
 2|Michael Armbrust| 1| [250, 100]| 1| Ph.D.| EECS|UC Berkeley|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+

In [8]:
# Outer joins evaluate the keys in both of the DataFrames or tables and includes (and joins together) the rows that evaluate to true or false. 
# If there is no equivalent row in either the left or right DataFrame, Spark will insert null:

In [9]:
joinType = "outer"

In [10]:
# Joining two DataFrames by joinType
person.join(graduateProgram, joinExpression, joinType).show(5)

+----+----------------+----------------+---------------+---+-------+--------------------+-----------+
 id| name|graduate_program| spark_status| id| degree| department| school|
+----+----------------+----------------+---------------+---+-------+--------------------+-----------+
 0| Bill Chambers| 0| [100]| 0|Masters|School of Informa...|UC Berkeley|
 1| Matei Zaharia| 1|[500, 250, 100]| 1| Ph.D.| EECS|UC Berkeley|
 2|Michael Armbrust| 1| [250, 100]| 1| Ph.D.| EECS|UC Berkeley|
null| null| null| null| 2|Masters| EECS|UC Berkeley|
+----+----------------+----------------+---------------+---+-------+--------------------+-----------+

In [11]:
# Left outer joins evaluate the keys in both of the DataFrames or tables and includes all rows from the left DataFrame as well as any rows in the right DataFrame that have a match in the left DataFrame. If there is no equivalent row in the right DataFrame, Spark will insert null:

In [12]:
# Specifying the join type as 'left_outer'
joinType = "left_outer"

In [13]:
# Performing the Left outer join.
graduateProgram.join(person, joinExpression, joinType).show()
# Notice the null values in Person DataFrame.

+---+-------+--------------------+-----------+----+----------------+----------------+---------------+
 id| degree| department| school| id| name|graduate_program| spark_status|
+---+-------+--------------------+-----------+----+----------------+----------------+---------------+
 0|Masters|School of Informa...|UC Berkeley| 0| Bill Chambers| 0| [100]|
 1| Ph.D.| EECS|UC Berkeley| 1| Matei Zaharia| 1|[500, 250, 100]|
 1| Ph.D.| EECS|UC Berkeley| 2|Michael Armbrust| 1| [250, 100]|
 2|Masters| EECS|UC Berkeley|null| null| null| null|
+---+-------+--------------------+-----------+----+----------------+----------------+---------------+

In [14]:
# Right outer joins evaluate the keys in both of the DataFrames or tables and includes all rows from the right DataFrame as well as any rows in the left DataFrame that have a match in the right DataFrame. If there is no equivalent row in the left DataFrame, Spark will insert null:

In [15]:
# Specifying the join type as right_outer
joinType = "right_outer"

In [16]:
# Notice the null values for Graduate_program DataFrames.
person.join(graduateProgram, joinExpression, joinType).show()

+----+----------------+----------------+---------------+---+-------+--------------------+-----------+
 id| name|graduate_program| spark_status| id| degree| department| school|
+----+----------------+----------------+---------------+---+-------+--------------------+-----------+
 0| Bill Chambers| 0| [100]| 0|Masters|School of Informa...|UC Berkeley|
 1| Matei Zaharia| 1|[500, 250, 100]| 1| Ph.D.| EECS|UC Berkeley|
 2|Michael Armbrust| 1| [250, 100]| 1| Ph.D.| EECS|UC Berkeley|
null| null| null| null| 2|Masters| EECS|UC Berkeley|
+----+----------------+----------------+---------------+---+-------+--------------------+-----------+

In [17]:
# Semi joins are a bit of a departure from the other joins. They do not actually include any values from the right DataFrame. 
# They only compare values to see if the value exists in the second DataFrame. 
# If the value does exist, those rows will be kept in the result, even if there are duplicate keys in the left DataFrame. 
# Think of left semi joins as filters on a DataFrame, as opposed to the function of a conventional join:

In [18]:
# Specifying the "left_semi" join
joinType = "left_semi"

In [19]:
# Performing the Left_semi join
graduateProgram.join(person, joinExpression, joinType).show()

# Notice the result DataFrame contains only Left DataFrame ie "graduateProgram" values.

+---+-------+--------------------+-----------+
 id| degree| department| school|
+---+-------+--------------------+-----------+
 0|Masters|School of Informa...|UC Berkeley|
 1| Ph.D.| EECS|UC Berkeley|
+---+-------+--------------------+-----------+

In [20]:
# Left anti joins are the opposite of left semi joins. 
# Like left semi joins, they do not actually include any values from the right DataFrame. They only compare values to see if the value exists in the second DataFrame. However, rather than keeping the values that exist in the second DataFrame, they keep only the values that do not have a corresponding key in the second DataFrame. 
# Think of anti joins as a NOT IN SQL-style filter:

In [21]:
# Specifying the "left_anti" join
joinType = "left_anti"

In [22]:
# Performing the "left_anti" join
graduateProgram.join(person, joinExpression, joinType).show()

# Notice the result DataFrame contains only Left DataFrame.

+---+-------+----------+-----------+
 id| degree|department| school|
+---+-------+----------+-----------+
 2|Masters| EECS|UC Berkeley|
+---+-------+----------+-----------+

In [23]:
# Natural joins make implicit guesses at the columns on which you would like to join. It finds matching columns and returns the results. Left, right, and outer natural joins are all supported.

In [24]:
# Cross-joins in simplest terms are inner joins that do not specify a predicate. 
# Cross joins will join every single row in the left DataFrame to ever single row in the right DataFrame. 
# This will cause an absolute explosion in the number of rows contained in the resulting DataFrame. 
# If you have 1,000 rows in each DataFrame, the cross-join of these will result in 1,000,000 (1,000 x 1,000) rows. 

In [25]:
# Specifying the join type as 'cross'
joinType = "cross"
# Performing the "cross join".
graduateProgram.join(person, joinExpression, joinType).show()
# Be careful when you cross join on large datasets, it causes the explosion in the number of rows contained in result DF.

+---+-------+--------------------+-----------+---+----------------+----------------+---------------+
 id| degree| department| school| id| name|graduate_program| spark_status|
+---+-------+--------------------+-----------+---+----------------+----------------+---------------+
 0|Masters|School of Informa...|UC Berkeley| 0| Bill Chambers| 0| [100]|
 1| Ph.D.| EECS|UC Berkeley| 1| Matei Zaharia| 1|[500, 250, 100]|
 1| Ph.D.| EECS|UC Berkeley| 2|Michael Armbrust| 1| [250, 100]|
+---+-------+--------------------+-----------+---+----------------+----------------+---------------+

In [26]:
# When performing joins, there are some specific challenges and some common questions that arise. 
# The rest of the notebook will provide answers to these common questions and then explain how, at a high level, Spark performs joins.

In [27]:
# Even though this might seem like a challenge, it’s actually not. Any expression is a valid join expression, assuming that it returns a Boolean:
# For ex: Joining by the id from the person DF and spark_status array from the SparkStatus DF.
from pyspark.sql.functions import expr
# As spark_status is an array column, using an expression to check if the spark_status contains an id value.
person.withColumnRenamed("id", "personId")\
  .join(sparkStatus, expr("array_contains(spark_status, id)")).show()

+--------+----------------+----------------+---------------+---+--------------+
personId| name|graduate_program| spark_status| id| status|
+--------+----------------+----------------+---------------+---+--------------+
 0| Bill Chambers| 0| [100]|100| Contributor|
 1| Matei Zaharia| 1|[500, 250, 100]|500|Vice President|
 1| Matei Zaharia| 1|[500, 250, 100]|250| PMC Member|
 1| Matei Zaharia| 1|[500, 250, 100]|100| Contributor|
 2|Michael Armbrust| 1| [250, 100]|250| PMC Member|
 2|Michael Armbrust| 1| [250, 100]|100| Contributor|
+--------+----------------+----------------+---------------+---+--------------+

In [28]:
# One of the tricky things that come up in joins is dealing with duplicate column names in your results DataFrame.
# In a DataFrame, each column has a unique ID within Spark’s SQL Engine, Catalyst. This unique ID is purely internal and not something that you can directly reference. 
# This makes it quite difficult to refer to a specific column when you have a DataFrame with duplicate column names.

In [29]:
# This can occur in two distinct situations:

# The join expression that you specify does not remove one key from one of the input DataFrames and the keys have the same column name
# Two columns on which you are not performing the join have the same name

In [30]:
# Let’s create a problem dataset that we can use to illustrate these problems:
gradProgramDupe = graduateProgram.withColumnRenamed("id", "graduate_program")

In [31]:
# Joining expression
joinExpr = gradProgramDupe["graduate_program"] == person["graduate_program"]

In [32]:
# Joining using the above joinExpression. By default its equi-join.
person.join(gradProgramDupe, joinExpr).show()

# Not below that the result DF contains duplicate graduate_program columns. ( as both DataFrames contains the same column name)

+---+----------------+----------------+---------------+----------------+-------+--------------------+-----------+
 id| name|graduate_program| spark_status|graduate_program| degree| department| school|
+---+----------------+----------------+---------------+----------------+-------+--------------------+-----------+
 0| Bill Chambers| 0| [100]| 0|Masters|School of Informa...|UC Berkeley|
 1| Matei Zaharia| 1|[500, 250, 100]| 1| Ph.D.| EECS|UC Berkeley|
 2|Michael Armbrust| 1| [250, 100]| 1| Ph.D.| EECS|UC Berkeley|
+---+----------------+----------------+---------------+----------------+-------+--------------------+-----------+

In [33]:
# The problem arises when you try to access one of the graduate_program column

person.join(gradProgramDupe, joinExpr).select("graduate_program").show()

--------------------------------------------------------------------------- 
 AnalysisException Traceback (most recent call last)
 <command-4112042119781543> in <module> () 
 1 # The problem arises when you try to access one of the graduate_program column 
 2 
 ----> 3 person . join ( gradProgramDupe , joinExpr ) . select ( "graduate_program" ) . show ( ) 

 /databricks/spark/python/pyspark/sql/dataframe.py in select (self, *cols) 
 1263 [ Row ( name = u'Alice' , age = 12 ) , Row ( name = u'Bob' , age = 15 ) ] 
 1264 """
 -> 1265 jdf = self . _jdf . select ( self . _jcols ( * cols ) ) 
 1266 return DataFrame ( jdf , self . sql_ctx ) 
 1267 

 /databricks/spark/python/lib/py4j-0.10.6-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1158 answer = self . gateway_client . send_command ( command ) 
 1159 return_value = get_return_value(
 -> 1160 answer, self.gateway_client, self.target_id, self.name)
 1161 
 1162 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 67 e.java_exception.getStackTrace()))
 68 if s . startswith ( 'org.apache.spark.sql.AnalysisException: ' ) : 
 ---> 69 raise AnalysisException ( s . split ( ': ' , 1 ) [ 1 ] , stackTrace ) 
 70 if s . startswith ( 'org.apache.spark.sql.catalyst.analysis' ) : 
 71 raise AnalysisException ( s . split ( ': ' , 1 ) [ 1 ] , stackTrace ) 

 AnalysisException : u"Reference 'graduate_program' is ambiguous, could be: graduate_program, graduate_program.;"

In [34]:
# APPROACH 1: DIFFERENT JOIN EXPRESSION

# When you have two keys that have the same name, probably the easiest fix is to change the join expression from a Boolean expression to a string or sequence. This automatically removes one of the columns for you during the join:

person.join(gradProgramDupe,"graduate_program").select("graduate_program").show()

+----------------+
graduate_program|
+----------------+
 0|
 1|
 1|
+----------------+

In [35]:
# APPROACH 2: DROPPING THE COLUMN AFTER THE JOIN
# Another approach is to drop the offending column after the join. When doing this, we need to refer to the column via the original source DataFrame.
# We can do this if the join uses the same key names or if the source DataFrames have columns that simply have the same name:

person.join(gradProgramDupe, joinExpr).drop(person["graduate_program"]).select("graduate_program").show()

+----------------+
graduate_program|
+----------------+
 0|
 1|
 1|
+----------------+

In [36]:
# APPROACH 3: RENAMING A COLUMN BEFORE THE JOIN
# We can avoid this issue altogether if we rename one of our columns before the join:

# Renaming the grad_id column to id.
gradProgram3 = graduateProgram.withColumnRenamed("id", "grad_id")
joinExpr = person["graduate_program"] == gradProgram3["grad_id"]
# Joining the DataFrames.
person.join(gradProgram3, joinExpr).show()

+---+----------------+----------------+---------------+-------+-------+--------------------+-----------+
 id| name|graduate_program| spark_status|grad_id| degree| department| school|
+---+----------------+----------------+---------------+-------+-------+--------------------+-----------+
 0| Bill Chambers| 0| [100]| 0|Masters|School of Informa...|UC Berkeley|
 1| Matei Zaharia| 1|[500, 250, 100]| 1| Ph.D.| EECS|UC Berkeley|
 2|Michael Armbrust| 1| [250, 100]| 1| Ph.D.| EECS|UC Berkeley|
+---+----------------+----------------+---------------+-------+-------+--------------------+-----------+

In [37]:
# Analysing the JOIN plan

joinExpr = person["graduate_program"] == graduateProgram["id"]
person.join(graduateProgram, joinExpr).explain()

# Notice the Exchange Hashpartitioning stage for the join, which represents the all node-to-node communication.

== Physical Plan ==
*(5) SortMergeJoin [graduate_program#12L], [id#26L], Inner
:- *(2) Sort [graduate_program#12L ASC NULLS FIRST], false, 0
: +- Exchange hashpartitioning(graduate_program#12L, 200)
: +- *(1) Project [_1#2L AS id#10L, _2#3 AS name#11, _3#4L AS graduate_program#12L, _4#5 AS spark_status#13]
: +- *(1) Filter isnotnull(_3#4L)
: +- *(1) Scan ExistingRDD[_1#2L,_2#3,_3#4L,_4#5]
+- *(4) Sort [id#26L ASC NULLS FIRST], false, 0
 +- Exchange hashpartitioning(id#26L, 200)
 +- *(3) Project [_1#18L AS id#26L, _2#19 AS degree#27, _3#20 AS department#28, _4#21 AS school#29]
 +- *(3) Filter isnotnull(_1#18L)
 +- *(3) Scan ExistingRDD[_1#18L,_2#19,_3#20,_4#21]

In [38]:
# With the DataFrame API, we can explicitly give the optimizer a hint that we would like to use a broadcast join by using the correct function around the small DataFrame in question.

from pyspark.sql.functions import broadcast
joinExpr = person["graduate_program"] == graduateProgram["id"]
person.join(broadcast(graduateProgram), joinExpr).explain()

# Notice the BroadcastHashJoin, which represents the Broadcast join.

== Physical Plan ==
*(2) BroadcastHashJoin [graduate_program#12L], [id#26L], Inner, BuildRight, false
:- *(2) Project [_1#2L AS id#10L, _2#3 AS name#11, _3#4L AS graduate_program#12L, _4#5 AS spark_status#13]
: +- *(2) Filter isnotnull(_3#4L)
: +- *(2) Scan ExistingRDD[_1#2L,_2#3,_3#4L,_4#5]
+- BroadcastExchange HashedRelationBroadcastMode(List(input[0, bigint, true]))
 +- *(1) Project [_1#18L AS id#26L, _2#19 AS degree#27, _3#20 AS department#28, _4#21 AS school#29]
 +- *(1) Filter isnotnull(_1#18L)
 +- *(1) Scan ExistingRDD[_1#18L,_2#19,_3#20,_4#21]

In [39]:
# Broadcast Join:
# When the table is small enough to fit into the memory of a single worker node, with some breathing room of course, we can optimize our join. 
# Spark Driver will replicate small DataFrame onto every worker node in the cluster (be it located on one machine or many). Now this sounds expensive. However, what this does is prevent us from performing the all-to-all communication during the entire join process. Instead, we perform it only once at the beginning and then let each individual worker node perform the work without having to wait or communicate with any other worker node.
# This greatly reduces the communication between node-node and optimizes the join.

In [40]:
# Also partition your data correctly prior to a join, so that you can end up with much more efficient execution because even if a shuffle is planned, if data from two different DataFrames is already located on the same machine, Spark can avoid the shuffle. 

In [41]:
# Congratulations!!

# You have learnt the different joins possible in Spark also you have learnt the different optimizations that can be done for Joins.